<a href="https://colab.research.google.com/github/arunkumarviswanathan/learning/blob/main/pdf_read.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# My First Jupyter Notebook

This is a sample Jupyter Notebook demonstrating basic functionality.

In [16]:
print("Hello, World!")

## installing required modules
!pip install PdfReader
!pip install langchain
!pip install PyPDF2
!pip install InstructorEmbedding
!pip install sentence_transformers
!pip install faiss
!pip install faiss-gpu
!pip install langchain-community


Hello, World!
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.0/320.0 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 48.2 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement faiss (from versions: none)
ERROR: No matching distribution found for faiss
ERROR: Could not find a version that satisfies the requirement faiss-gpu (from versions: none)
ERROR: No matching distribution found for faiss-gpu


In [17]:
## import required libraries
from PyPDF2 import PdfReader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, HuggingFaceInstructEmbeddings
from langchain.vectorstores import FAISS
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFaceHub
import os
from langchain.prompts.prompt import PromptTemplate

In [18]:
## extracting text from pdf files
def get_pdf_text(pdf_docs):
    text = ""
    for pdf in pdf_docs:
        pdf_reader = PdfReader(pdf)
        for page in pdf_reader.pages:
            text += page.extract_text()
    return text## creating overlapping text chunks
def get_text_chunks(text):
    text_splitter = CharacterTextSplitter(
        separator="\n",
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len
    )
    chunks = text_splitter.split_text(text)
    return chunks## creating embeddings for chunks of text

def get_vectorstore(text_chunks):
    #embeddings = OpenAIEmbeddings()
    embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-xl")
    vectorstore = FAISS.from_texts(texts=text_chunks, embedding=embeddings)
    return vectorstore## ceating a retrival llm chain

def retrieval_qa_chain(db,return_source_documents):
    llm = HuggingFaceHub(repo_id="tiiuae/falcon-7b-instruct", model_kwargs={"temperature":0.6,"max_length":500, "max_new_tokens":700})
    qa_chain = RetrievalQA.from_chain_type(llm=llm,
                                       chain_type='stuff',
                                       retriever=db,
                                       return_source_documents=return_source_documents,
                                       )
    return qa_chain

In [19]:
from google.colab import files
uploaded = files.upload()
path_to_pdf = uploaded.keys()
for key in uploaded.keys():
  print("Key: " + key)

## DATA VECTORIZATION AND INDEX CREATION


# get the text chunks
raw_text = get_pdf_text(path_to_pdf)
# create vector store
text_chunks = get_text_chunks(raw_text)
## creating a db with similarity search and obtaining top 3 most matched vectors of all the vectors present in vector index
vectorstore = get_vectorstore(text_chunks)
db = vectorstore.as_retriever(search_kwargs={'k': 3})

## passing database to bot as input and initializing the bot
bot = retrieval_qa_chain(db,True)

Saving UpdateGuide.pdf to UpdateGuide (1).pdf
Key: UpdateGuide (1).pdf


/tmp/ipython-input-18-1708696138.py:21: LangChainDeprecationWarning: The class `HuggingFaceInstructEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-xl")
/usr/local/lib/python3.11/dist-packages/InstructorEmbedding/instructor.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab

modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

TypeError: INSTRUCTOR._load_sbert_model() got an unexpected keyword argument 'token'

In [ ]:
## passing query to llm
query = "what is  Nampdicta?"
sol=bot(query)
